## Imports

In [49]:
from utility.utility_general import *
from utility import utility_crystal
import imp, crystals_tools
imp.reload(crystals_tools)

from sklearn.model_selection import train_test_split
import h5py

# pd.set_option('display.width', 1000)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_rows',None)
# pd.set_option('display.max_colwidth', 500)

from datetime import datetime
from joblib import Parallel, delayed
from tqdm import tqdm, tqdm_notebook

## Loading CIF files from the banks

In [4]:
# top_n = 300

pos_files = list_all_files(f'{local_data_path}/data_banks/cod/',
                           pattern='**/*.cif')
# neg_files = list_all_files(f'{local_data_path}/data_banks/cspd_{top_n}/',
#                            pattern='**/*.cif')
neg_files = list_all_files(f'{local_data_path}/data_banks/',
                           pattern='cspd_*/**/*.cif')

pos_df = pd.DataFrame({
    'id': [i.split('/')[-1].split('.')[0] for i in pos_files],
    'db': 'cod',
    'y': [1] * len(pos_files),
    'cif_file': pos_files,
})
neg_df = pd.DataFrame({
    'id': [(i.split('/')[-1].split('.')[0]) for i in neg_files],
    'db': [i.split('/')[-3] for i in neg_files],
    'y': [-1] * len(neg_files),
    'cif_file': neg_files,
})
df = pd.concat([pos_df, neg_df]).reset_index(drop=True)
df['img_path'] = df['cif_file'].str.replace('Local_Data/', 'Data/') + '.npy'
df_all = df.copy()


df = df[df['y']==1]
# df = pos_df
df

,id,db,y,cif_file,img_path
0,1000000,cod,1,/home/adavar2/Local_Data/syn/data_banks/cod/ci...,/home/adavar2/Data/syn/data_banks/cod/cif/1/00...
1,1000001,cod,1,/home/adavar2/Local_Data/syn/data_banks/cod/ci...,/home/adavar2/Data/syn/data_banks/cod/cif/1/00...
2,1000002,cod,1,/home/adavar2/Local_Data/syn/data_banks/cod/ci...,/home/adavar2/Data/syn/data_banks/cod/cif/1/00...
3,1000003,cod,1,/home/adavar2/Local_Data/syn/data_banks/cod/ci...,/home/adavar2/Data/syn/data_banks/cod/cif/1/00...
4,1000004,cod,1,/home/adavar2/Local_Data/syn/data_banks/cod/ci...,/home/adavar2/Data/syn/data_banks/cod/cif/1/00...
...,...,...,...,...,...
285616,9016520_super,cod,1,/home/adavar2/Local_Data/syn/data_banks/cod/ci...,/home/adavar2/Data/syn/data_banks/cod/cif_supe...
285617,9016605_super,cod,1,/home/adavar2/Local_Data/syn/data_banks/cod/ci...,/home/adavar2/Data/syn/data_banks/cod/cif_supe...
285618,9016621_super,cod,1,/home/adavar2/Local_Data/syn/data_banks/cod/ci...,/home/adavar2/Data/syn/data_banks/cod/cif_supe...
285619,9016666_super,cod,1,/home/adavar2/Local_Data/syn/data_banks/cod/ci...,/home/adavar2/Data/syn/data_banks/cod/cif_supe...


In [20]:
neg_df

,id,db,y,cif_file
0,0000000,cspd_108,-1,/home/adavar2/Local_Data/syn/data_banks/cspd_1...
1,0000001,cspd_108,-1,/home/adavar2/Local_Data/syn/data_banks/cspd_1...
2,0000002,cspd_108,-1,/home/adavar2/Local_Data/syn/data_banks/cspd_1...
3,0000003,cspd_108,-1,/home/adavar2/Local_Data/syn/data_banks/cspd_1...
4,0000004,cspd_108,-1,/home/adavar2/Local_Data/syn/data_banks/cspd_1...
...,...,...,...,...
2141,2990000,cspd_300,-1,/home/adavar2/Local_Data/syn/data_banks/cspd_3...
2142,2990001,cspd_300,-1,/home/adavar2/Local_Data/syn/data_banks/cspd_3...
2143,2990002,cspd_300,-1,/home/adavar2/Local_Data/syn/data_banks/cspd_3...
2144,2990003,cspd_300,-1,/home/adavar2/Local_Data/syn/data_banks/cspd_3...


## Converting CIF files to images
### Removing the old positive images

In [45]:
pos_imgs = list_all_files(f'{data_path}/data_banks/cod/',
                           pattern='**/*.npy')

pos_imgs = pd.Series(pos_imgs)
to_del = pd.Series(list(set(pos_imgs) - set(df['img_path'])))
to_conv2img = pd.Series(list(set(df['img_path']) - set(pos_imgs)))

print('Te be deleted: {:,} - To be converted to image: {:,}'.format(len(to_del), len(to_conv2img)))
# for f in to_del:
#     os.remove(f)

df = df[df['img_path'].isin(to_conv2img)].reset_index(drop=True)

Te be deleted: 0 - To be converted to image: 0


/home/adavar2/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """
/home/adavar2/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


### Removing the negative set images
It's better do be done manually:
~~~
cd ~/Data/syn/data_banks/cspd_108/
rm -r 0*
~~~

### Creating new images

In [21]:
time_1 = datetime.now()

output = Parallel(n_jobs=64)(delayed(crystals_tools.cif2image)
#                              (row['cif_file'], save_to=row['img_path'], create_path=True, ok_error=True,
                             (df['cif_file'][i], save_to=df['img_path'][i], create_path=True, ok_error=True,
                             skip_min_atomic_dist=True) 
#                              for _, row in tqdm_notebook(df.iterrows()))
                             for i, _ in tqdm_notebook(df.iterrows(), total=len(df)))

time_2 = datetime.now()
print('Finished: Creating anomalies. Run time:', str(time_2 - time_1)[:-7])

/home/adavar2/anaconda3/envs/explain/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning



Finished: Creating anomalies. Run time: 0:03:00


In [140]:
time_1 = datetime.now()

output = Parallel(n_jobs=64)(delayed(crystals_tools.cif2image)
#                              (row['cif_file'], save_to=row['img_path'], create_path=True, ok_error=True,
                             (df['cif_file'][i], save_to=df['img_path'][i], create_path=True, ok_error=False,
                             skip_min_atomic_dist=True) 
#                              for _, row in tqdm_notebook(df.iterrows()))
                             for i in tqdm_notebook(range(len(df))))

time_2 = datetime.now()
print('Finished: Creating anomalies. Run time:', str(time_2 - time_1)[:-7])

/home/adavar2/anaconda3/envs/explain/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Finished: Creating anomalies. Run time: 0:03:00


### Checking if all the cif files in the bank were converted to images or not

In [22]:
# Checking if all the cif files in the bank were converted to images or not
# img_files = list_all_files(f'{data_path}/data_banks/', pattern='**/*.npy')
img_files = list_all_files(f'{data_path}/data_banks/', pattern='cspd_*/**/*.npy')
print(len(img_files), len(df))
# assert len(img_files) == len(df)

2146 2146


## Creating data sets

### Data set 9:
The same as method as data set 7 with a constraint on min_dist >0.9

In [90]:
from ase.formula import Formula
path = f'{data_path}/data_sets/data_set_9/'
top_n = 108

df_neg = df_all[df_all['db'] == f'cspd_{top_n}'].sample(frac=1, random_state=0)
df = pd.read_csv(f'{local_data_path}/data_bases/cspd/cif_top_{top_n}/df.csv')
df['id'] = [i.split('/')[-1].split('.')[0] for i in df['file']]
assert df['id'].isin(df_neg['id']).all()
df_neg = pd.merge(df_neg, df[['id', 'sgn', 'symbols']], on='id', how='left')
df_neg['formula_hill'] = [Formula(i).reduce()[0].format('hill') for i in df_neg['symbols']]

df_pos = pd.read_csv(f'{local_data_path}/data_banks/cod/df.csv', index_col=0)
df_pos['cif_file'] = local_data_path + '/data_banks/cod' + df_pos['filename'].astype('string')
# from utility import utility_crystal
# sg = utility_crystal.SpaceGroups()
# df_pos['sgn'] = [sg.convert(i, 'Hall', 'sgn', pick_one=True) for i in df_pos['spacegroup']]
df_pos['formula_hill'] = [Formula(i).reduce()[0].format('hill') for i in df_pos['formula']]

In [91]:
n_df_pos_initial = len(df_pos)
df_pos = df_pos[df_pos['min_atomic_dist'] > (3**.5 * 70/128)]
print(f'Number of data positive data points fitting in the resolution constraint: {len(df_pos):,}/{n_df_pos_initial:,}')

Number of data positive data points fitting in the resolution constraint: 111,461/285,621


In [92]:
df = df_pos[df_pos['formula_hill'].isin(df_neg['formula_hill'])]
print('Positive crystals with sharing the same compositions as negatives: {:,}'.format(len(df)))
df_pos_1 = df_all[df_all['cif_file'].isin(df['cif_file'])]
assert len(df) == len(df_pos_1)
df = df_all[~df_all['cif_file'].isin(df_pos_1['cif_file'])]
assert len(df) + len(df_pos_1) == len(df_all)
df_pos_2 = df[df['db'] == 'cod'].sample(n=len(df_neg) * 5 - len(df_pos_1), random_state=0)
df_pos = pd.concat([df_pos_1, df_pos_2])
assert len(df_pos) == len(df_neg) * 5

df_ds = pd.concat([df_neg, df_pos]).sample(frac=1, random_state=0)
df_train, df_test = train_test_split(df_ds, test_size=0.3, random_state=0)
df_train, df_dev = train_test_split(df_train, test_size=0.3, random_state=0)

print('Tot. samples={:,}  [Pos.={:,} - Neg.={:,}({:.1f}%)]'.format(
    len(df_ds), len(df_ds[df_ds['y']==1]), len(df_ds[df_ds['y']==-1]), 
    len(df_ds[df_ds['y']==-1])/len(df_ds)*100
))

print('Tot. train={:,} - Tot. dev.={:,} - Tot. test={:,}'.format(
    len(df_train), len(df_dev), len(df_test)
))

# makedirs(path, exist_ok=True)
# df_train.to_csv(path + 'train.csv')
# df_test.to_csv(path + 'test.csv')
# df_dev.to_csv(path + 'dev.csv')

Positive crystals with sharing the same compositions as negatives: 367
Tot. samples=3,600  [Pos.=3,000 - Neg.=600(16.7%)]
Tot. train=1,764 - Tot. dev.=756 - Tot. test=1,080


### Data set of electrode and thermoelectric materials

In [66]:
thermo = pd.read_csv(f'{data_path}/thermoelectric_materials/thermoelectric_materials.csv')

thermo.rename(columns={'data_set': 'db', 'filename': 'file'}, inplace=True)
assert ~ thermo.duplicated(subset=['file']).any()
thermo

,year,formula,db,file,sgn,HM,formula_hill
0,2001,EuH3Li,mp,/home/ali/Data/cod/thermoelectric/cif-top-10s/...,221.0,Pm-3m,H3EuLi
1,2001,CdGeP2,mp-avail-in-cod,/home/ali/Data/cod/thermoelectric/cif-top-10s/...,122.0,I-42d,CdGeP2
2,2001,CdGeP2,mp,/home/ali/Data/cod/thermoelectric/cif-top-10s/...,225.0,Fm-3m,CdGeP2
3,2001,CdGeP2,mp,/home/ali/Data/cod/thermoelectric/cif-top-10s/...,160.0,R3m,CdGeP2
4,2001,CdGeP2,cod,/home/ali/Data/cod/thermoelectric/cif-top-10s/...,122.0,I -4 2 d,CdGeP2
...,...,...,...,...,...,...,...
209,2018,CuTe,mp-avail-in-cod-duplicated,/home/ali/Data/cod/thermoelectric/cif-top-10s/...,59.0,Pmmn,CuTe
210,2018,CuTe,mp-avail-in-cod,/home/ali/Data/cod/thermoelectric/cif-top-10s/...,59.0,Pmmn,CuTe
211,2018,CuTe,cod,/home/ali/Data/cod/thermoelectric/cif-top-10s/...,59.0,P m m n,CuTe
212,2018,CuTe,cod-duplicated,/home/ali/Data/cod/thermoelectric/cif-top-10s/...,59.0,P m m n,CuTe


In [79]:
# df = pd.read_csv('tmp/electrode-and-thermo-df.csv', index_col=0)
# df
# thermo = load_var(f'{data_path}/thermoelectric_materials/atoms_list_top_10.pkl')
thermo = pd.read_csv(f'{data_path}/thermoelectric_materials/thermoelectric_materials.csv')

thermo.rename(columns={'data_set': 'db', 'filename': 'file'}, inplace=True)
assert ~ thermo.duplicated(subset=['file']).any()
thermo['type'] = 'thermoelectric'
thermo['y'] = 1
thermo['file'] = thermo['file'].str.replace('/ali/', '/adavar2/')
assert np.array([exists(f) for f in thermo['file']]).all() # File existance check
thermo = thermo[['type', 'db', 'y', 'file']]

electrode_df = pd.read_csv(data_path + '/electrode_materials/electrode-materials.csv')
electrode_df['type'] = 'electrode'
electrode_df['y'] = 1
electrode_df['file'] = electrode_df['path']
electrode_df.loc[electrode_df['db']=='cod-sc', 'db'] = 'cod'
electrode_df = electrode_df[list(thermo.columns)]

# thermo
# electrode_df
df = pd.concat([electrode_df, thermo]).reset_index(drop=True)
# df['img_path'] = [f'{data_path}/tmp/electrode-thermo-img/{i//100:03}/{3:04}.npy' for i in range(len(df))]
df['img_path'] = [f'{data_path}/data_banks/electrode-thermoelectric_materials/imgs/{i//100:03}/{i:05}.npy'\
                  for i in range(len(df))]

df.loc[~np.array([exists(f) for f in df['file']]), 'file'] = local_data_path +\
    df.loc[~np.array([exists(f) for f in df['file']]), 'file']

assert np.array([exists(f) for f in df['file']]).all()
assert np.array([exists(f) for f in df['img_path']]).all()
assert ~df.duplicated(subset=['file']).any()
assert ~df.duplicated(subset=['img_path']).any()

# df.to_csv(data_path + '/data_sets/electrode_thermoelectric_materials.csv', index=False)
df

,type,db,y,file,img_path
0,electrode,mp,1,/home/adavar2/Data/syn/electrode_materials/dow...,/home/adavar2/Data/syn/data_banks/electrode-th...
1,electrode,mp,1,/home/adavar2/Data/syn/electrode_materials/dow...,/home/adavar2/Data/syn/data_banks/electrode-th...
2,electrode,mp,1,/home/adavar2/Data/syn/electrode_materials/dow...,/home/adavar2/Data/syn/data_banks/electrode-th...
3,electrode,mp,1,/home/adavar2/Data/syn/electrode_materials/dow...,/home/adavar2/Data/syn/data_banks/electrode-th...
4,electrode,mp,1,/home/adavar2/Data/syn/electrode_materials/dow...,/home/adavar2/Data/syn/data_banks/electrode-th...
...,...,...,...,...,...
2564,thermoelectric,mp-avail-in-cod-duplicated,1,/home/adavar2/Data/cod/thermoelectric/cif-top-...,/home/adavar2/Data/syn/data_banks/electrode-th...
2565,thermoelectric,mp-avail-in-cod,1,/home/adavar2/Data/cod/thermoelectric/cif-top-...,/home/adavar2/Data/syn/data_banks/electrode-th...
2566,thermoelectric,cod,1,/home/adavar2/Data/cod/thermoelectric/cif-top-...,/home/adavar2/Data/syn/data_banks/electrode-th...
2567,thermoelectric,cod-duplicated,1,/home/adavar2/Data/cod/thermoelectric/cif-top-...,/home/adavar2/Data/syn/data_banks/electrode-th...


In [78]:
# df = pd.read_csv(data_path + '/data_sets/electrode_thermoelectric_materials.csv')
df.loc[df['type'] == 'thermoelectric', 'db'].value_counts()

mp                            66
cod                           56
mp-avail-in-cod               43
cod-duplicated                42
mp-avail-in-cod-duplicated     7
Name: db, dtype: int64

#### Creating images

In [214]:
time_1 = datetime.now()
shutil.rmtree(f'{data_path}/tmp/electrode-thermo-img/')

output = Parallel(n_jobs=64)(delayed(crystals_tools.cif2image)
                             (row['file'], save_to=row['img_path'], create_path=True, ok_error=True,
                             skip_min_atomic_dist=True) 
                             for _, row in tqdm_notebook(df.iterrows(), total=len(df)))

time_2 = datetime.now()
print('Finished: Creating anomalies. Run time:', str(time_2 - time_1)[:-7])

/home/adavar2/anaconda3/envs/explain/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning



Finished: Creating anomalies. Run time: 0:02:22


In [ ]:
# Checking if all the images are converted 
created_imgs = set(list_all_files(f'{data_path}/tmp/electrode-thermo-img/' ,pattern='**/*.npy'))
assert set(df['img_path']) == created_imgs